<a href="https://colab.research.google.com/github/Kyouma960/ChatBot/blob/main/llama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
!pip install nest_asyncio
!pip install discord.py

In [ ]:
!git clone https://github.com/tloen/alpaca-lora && cd alpaca-lora && pip install -r requirements.txt

In [ ]:
!pip install torch==2.0.0

In [ ]:
!nvidia-smi

In [ ]:
import torch
from peft import PeftModel
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig
from transformers.generation.utils import GreedySearchDecoderOnlyOutput

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("decapoda-research/llama-7b-hf")

In [ ]:
model = LlamaForCausalLM.from_pretrained(
  "decapoda-research/llama-7b-hf",
  load_in_8bit=True,
  device_map="auto"
)

In [ ]:
model = PeftModel.from_pretrained(
    model,
    "tloen/alpaca-lora-7b",
    torch_dtype=torch.float16
)

In [ ]:
model.config.pad_token_id = tokenizer.pad_token_id =0
model.config.bos_token_id = 1
model.config.eos_token_id = 2

In [ ]:
model = model.eval()
model = torch.compile(model)

In [ ]:
PROMPT_TEMPLATE = f"""
Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
[INSTRUCTION]

### Response:
"""

In [ ]:
PROMPT_INPUT_TEMPLATE = f"""
Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
[INSTRUCTION]

### Input:
[INPUT]

### Response:
"""

In [ ]:
def create_prompt(instruction: str) -> str:
  return PROMPT_TEMPLATE.replace("[INSTRUCTION]", instruction)

In [ ]:
def generate_response(prompt: str, model: PeftModel) -> GreedySearchDecoderOnlyOutput:
  encoding = tokenizer(prompt, return_tensors="pt")
  input_ids = encoding["input_ids"].to(DEVICE)
  generation_config = GenerationConfig(
    temperature = 0.1,
    top_p=0.75,
    repetition_penalty=1.1
  )
  with torch.inference_mode():
      global output
      output = model.generate(
          input_ids=input_ids,
          generation_config=generation_config,
          return_dict_in_generate=True,
          output_scores=True,
          max_new_tokens=1024
      )
      return output

In [ ]:
def format_response(response: GreedySearchDecoderOnlyOutput) -> str:
  decoded_output = tokenizer.decode(output.sequences[0])
  response = decoded_output.split("### Response:")[1].strip()
  return "\n".join(textwrap.wrap(response))

In [ ]:
def ask_alpaca(prompt: str, model: PeftModel = model) -> str:
  prompt = create_prompt(prompt)
  response = generate_response(prompt, model)
  return (format_response(response))

In [ ]:
#@markdown #**Anti-Disconnect for Google Colab**
#@markdown ## Run this to stop it from disconnecting automatically
#@markdown  **(It will anyhow disconnect after 6 - 12 hrs for using the free version of Colab.)**
#@markdown  *(Colab Pro users will get about 24 hrs usage time)*
#@markdown ---

import IPython
js_code = '''
function ClickConnect(){
console.log("Working");
document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect,60000)
'''
display(IPython.display.Javascript(js_code))

In [ ]:
import discord
from discord.ext import commands
import nest_asyncio
nest_asyncio.apply()
TOKEN = ''
intents = discord.Intents.default()
intents.message_content = True

client = commands.Bot(command_prefix='reg ', intents=intents)
@client.event
async def on_ready():
    print('bot ready')

@client.event
async def on_message(message):
    if message.author == client.user:
        return

    if message.content.startswith('reg ask '):
        user_message = message.content[8:]
        answer = ask_alpaca(user_message)
        embedVar = discord.Embed(title=user_message, description=answer, color=0x00ff00)
        await message.channel.send(embed=embedVar)
        #await message.channel.send(answer)

client.run(TOKEN)
